In [1]:
import os 
import sys 
sys.path.append("../training")
import pickle

import hashlib
import hls4ml 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score

import keras 
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, Softmax
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
import tensorflow_model_optimization as tfmot

from qkeras.qlayers import QDense, QActivation
from qkeras import QBatchNormalization
from qkeras.quantizers import quantized_bits, quantized_relu
from qkeras.utils import _add_supported_quantized_objects
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

2024-08-21 13:23:33.453672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 13:23:33.555062: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-21 13:23:33.555080: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-21 13:23:34.207495: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

WARN: Unable to import optimizer(s) from expr_templates.py: No module named 'sympy'


/home/jcampos/miniforge3/envs/ml4qick-env/lib/python3.8/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


## Load Data

#### Impoartant! 
Download the dataset locally from [OneDrive here](https://purdue0-my.sharepoint.com/personal/du245_purdue_edu/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fdu245%5Fpurdue%5Fedu%2FDocuments%2FShared%2FQSC%20ML%20for%20readout%2FFinal%5Fraw%5Fdata%5Ffor%5Fpaper%2Fdata%5F0528%5Fnpy). We are using QICK data with timestamp **0528**. 

In [2]:
def one_hot_encode(data):
    y_encoded = np.zeros([data.shape[0],2], dtype=np.int32)
    for idx, x in enumerate(data):
        if x == 1:
            y_encoded[idx][1] = 1
        else:
            y_encoded[idx][0] = 1
    return y_encoded

In [3]:
"""Loadning training split"""
start_window = 0
end_window = 770
data_dir = "../data/malab_05282024/npz/"
assert os.path.exists(f"{data_dir}/0528_X_train_{start_window}_{end_window}.npy"), "File does not exist "

x_train_path = os.path.join(data_dir, f'0528_X_train_{start_window}_{end_window}.npy')
y_train_path = os.path.join(data_dir, f'0528_y_train_{start_window}_{end_window}.npy')

X_train_val = np.load(x_train_path)
y_train_val = np.load(y_train_path)

# Insure same dataset is loaded 
assert hashlib.md5(X_train_val).hexdigest() == 'b61226c86b7dee0201a9158455e08ffb',  "Checksum failed. Wrong file was loaded or file may be corrupted."
assert hashlib.md5(y_train_val).hexdigest() == 'c59ce37dc7c73d2d546e7ea180fa8d31',  "Checksum failed. Wrong file was loaded or file may be corrupted."

y_train_val = one_hot_encode(y_train_val)

print("Train Data Set:")
print("\tX Path        :", x_train_path)
print("\ty Path        :", y_train_path)
print("\tSize          :", len(X_train_val))
print("\tSample Shape  :", X_train_val[0].shape)
print("\tMean          :", X_train_val.mean())
print("\tStd. Dev.     :", X_train_val.std())

assert len(X_train_val[0]) == (end_window-start_window)*2, "ERROR: Specified window does not match loaded dataset shape"

Train Data Set:
	X Path        : ../data/malab_05282024/npz/0528_X_train_0_770.npy
	y Path        : ../data/malab_05282024/npz/0528_y_train_0_770.npy
	Size          : 900000
	Sample Shape  : (1540,)
	Mean          : 57.37779754545455
	Std. Dev.     : 844.0956096913322


In [4]:
"""Loading testing split"""
start_window = 0
end_window = 770
data_dir = "../data/malab_05282024/npz/"
assert os.path.exists(f"{data_dir}/X_test_{start_window}_{end_window}.npy"), "File does not exist "

x_test_path = os.path.join(data_dir, f'0528_X_test_{start_window}_{end_window}.npy')
y_test_path = os.path.join(data_dir, f'0528_y_test_{start_window}_{end_window}.npy')

X_test = np.load(x_test_path)
y_test = np.load(y_test_path)

# Insure same dataset is loaded 
assert hashlib.md5(X_test).hexdigest() == 'b7d85f42522a0a57e877422bc5947cde', "Checksum failed. Wrong file was loaded or file may be corrupted."
assert hashlib.md5(y_test).hexdigest() == '8c9cce1821372380371ade5f0ccfd4a2', "Checksum failed. Wrong file was loaded or file may be corrupted."

y_test = one_hot_encode(y_test)

print("Test Data Set:")
print("\tX Path        :", x_test_path)
print("\ty Path        :", y_test_path)
print("\tSize         :", len(X_test))
print("\tSample Shape :", X_test[0].shape)
print("\tSample Shape :", X_test.mean())
print("\tStd. Dev.    :", X_test.std())

assert len(X_test[0]) == (end_window-start_window)*2, "ERROR: Specified window does not match loaded dataset shape"

Test Data Set:
	X Path        : ../data/malab_05282024/npz/0528_X_test_0_770.npy
	y Path        : ../data/malab_05282024/npz/0528_y_test_0_770.npy
	Size         : 100000
	Sample Shape : (1540,)
	Sample Shape : 57.57549828571429
	Std. Dev.    : 845.6158899866076


## Baseline Multi-layer Model 

<!-- ![Multi-layer model](../images/multi_layer_model.png) -->
<img src="../images/multi_layer_model.png" alt="alt text" width="75%">

In [79]:
"""hyperparameters"""
init_learning_rate = 1e-4
validation_split = 0
batch_size = 8192
epochs = 25
checkpoint_filename = "multi-layer.h5"
input_shape = (len(X_train_val[0]),)
start_window = 0
end_window = 770


In [80]:
sr = int((end_window-start_window)*2)
hn = sr * 2

model = Sequential()
model.add(Dense(int(hn/8), activation='relu', input_shape=(sr,), kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4),))
model.add(BatchNormalization())
model.add(Dense(2, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4)))

print(model.summary())
assert model.count_params() == 595597, 'Error. Total parameters has changed.'

baseline_param_count = model.count_params()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 385)               593285    
                                                                 
 batch_normalization_24 (Bat  (None, 385)              1540      
 chNormalization)                                                
                                                                 
 dense_38 (Dense)            (None, 2)                 772       
                                                                 
Total params: 595,597
Trainable params: 594,827
Non-trainable params: 770
_________________________________________________________________
None


### Train the model 

In [81]:
callbacks = [
        ModelCheckpoint(
        checkpoint_filename,
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        save_freq="epoch",
    ),
    ReduceLROnPlateau(patience=5, min_delta=1**-6)
]

In [82]:
opt = Adam(learning_rate=init_learning_rate)
model.compile(
    optimizer=opt, 
    loss=CategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

history = model.fit(
    X_train_val, 
    y_train_val, 
    batch_size=batch_size,
    epochs=epochs, 
    validation_split=0.05,   # 45,000 samples
    shuffle=True, 
    callbacks=callbacks,
)

Epoch 1/25
105/105 [==============================] - 4s 32ms/step - loss: 0.2967 - accuracy: 0.9298 - val_loss: 0.2701 - val_accuracy: 0.9474 - lr: 1.0000e-04
Epoch 2/25
105/105 [==============================] - 3s 29ms/step - loss: 0.2341 - accuracy: 0.9574 - val_loss: 0.2396 - val_accuracy: 0.9536 - lr: 1.0000e-04
Epoch 3/25
105/105 [==============================] - 3s 29ms/step - loss: 0.2222 - accuracy: 0.9598 - val_loss: 0.2273 - val_accuracy: 0.9574 - lr: 1.0000e-04
Epoch 4/25
105/105 [==============================] - 3s 29ms/step - loss: 0.2138 - accuracy: 0.9612 - val_loss: 0.2199 - val_accuracy: 0.9590 - lr: 1.0000e-04
Epoch 5/25
105/105 [==============================] - 3s 29ms/step - loss: 0.2073 - accuracy: 0.9618 - val_loss: 0.2153 - val_accuracy: 0.9593 - lr: 1.0000e-04
Epoch 6/25
105/105 [==============================] - 3s 29ms/step - loss: 0.2018 - accuracy: 0.9623 - val_loss: 0.2120 - val_accuracy: 0.9597 - lr: 1.0000e-04
Epoch 7/25
105/105 [====================

### Evaluate 

In [87]:
co = {}
_add_supported_quantized_objects(co)
model = load_model(checkpoint_filename, custom_objects=co, compile=False)

In [84]:
y_pred = model.predict(X_test)
print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))))

3125/3125 [==============================] - 4s 1ms/step
Keras  Accuracy: 0.96094


In [85]:
# get ground and excited indices 
e_indices = np.where(np.argmax(y_test, axis=1) == 1)[0]
g_indices = np.where(np.argmax(y_test, axis=1) == 0)[0]

# separate ground and excited samples 
Xe_test = X_test[e_indices]
ye_test = np.argmax(y_test, axis=1)[e_indices]

Xg_test = X_test[g_indices]
yg_test = np.argmax(y_test, axis=1)[g_indices]


In [86]:
# compute total correct for excited state 
ye_pred = model.predict(Xe_test)
e_accuracy = accuracy_score(ye_test, np.argmax(ye_pred, axis=1))

total_correct = (ye_test==np.argmax(ye_pred, axis=1)).astype(np.int8).sum()
total_incorrect = (ye_test!=np.argmax(ye_pred, axis=1)).astype(np.int8).sum()

print("Total correct:", total_correct)
print("Total incorrect:", total_incorrect)
print("Total samples:", len(Xe_test) )
print("Keras Excited Accuracy: {}".format(e_accuracy))

# compute total correct for ground state 
yg_pred = model.predict(Xg_test)
g_accuracy = accuracy_score(yg_test, np.argmax(yg_pred, axis=1))

total_correct = (yg_test==np.argmax(yg_pred, axis=1)).astype(np.int8).sum()
total_incorrect = (yg_test!=np.argmax(yg_pred, axis=1)).astype(np.int8).sum()

print("Total correct:", total_correct)
print("Total incorrect:", total_incorrect)
print("Total samples:", len(Xg_test) )
print("Keras Ground Accuracy: {}".format(g_accuracy))

# compute fidelity 
baseline_fidelity = 1-(1-g_accuracy)-(1-e_accuracy)
print('\n===================================')
print('Fidelity', baseline_fidelity)
print('===================================')

1563/1563 [==============================] - 3s 2ms/step
Total correct: 47360
Total incorrect: 2640
Total samples: 50000
Keras Excited Accuracy: 0.9472
1563/1563 [==============================] - 3s 2ms/step
Total correct: 48734
Total incorrect: 1266
Total samples: 50000
Keras Ground Accuracy: 0.97468

Fidelity 0.92188


## Explore Larger Models


In [73]:
def compute_fidelity(X_data, y_data):
    # get ground and excited indices 
    e_indices = np.where(np.argmax(y_data, axis=1) == 1)[0]
    g_indices = np.where(np.argmax(y_data, axis=1) == 0)[0]

    # separate ground and excited samples 
    Xe_test = X_data[e_indices]
    ye_test = np.argmax(y_data, axis=1)[e_indices]

    Xg_test = X_data[g_indices]
    yg_test = np.argmax(y_data, axis=1)[g_indices]

    # compute total correct for excited state 
    ye_pred = model.predict(Xe_test)
    e_accuracy = accuracy_score(ye_test, np.argmax(ye_pred, axis=1))

    total_correct = (ye_test==np.argmax(ye_pred, axis=1)).astype(np.int8).sum()
    total_incorrect = (ye_test!=np.argmax(ye_pred, axis=1)).astype(np.int8).sum()

    print("Total correct:", total_correct)
    print("Total incorrect:", total_incorrect)
    print("Total samples:", len(Xe_test) )
    print("Keras Excited Accuracy: {}".format(e_accuracy))

    # compute total correct for ground state 
    yg_pred = model.predict(Xg_test)
    g_accuracy = accuracy_score(yg_test, np.argmax(yg_pred, axis=1))

    total_correct = (yg_test==np.argmax(yg_pred, axis=1)).astype(np.int8).sum()
    total_incorrect = (yg_test!=np.argmax(yg_pred, axis=1)).astype(np.int8).sum()

    print("Total correct:", total_correct)
    print("Total incorrect:", total_incorrect)
    print("Total samples:", len(Xg_test) )
    print("Keras Ground Accuracy: {}".format(g_accuracy))

    # compute fidelity 
    fidelity = 1-(1-g_accuracy)-(1-e_accuracy)
    # print('\n===================================')
    # print('Fidelity', fidelity)
    
    # print('===================================')
    return fidelity


In [102]:
"""hyperparameters"""
init_learning_rate = 1e-3
validation_split = 0
batch_size = 8192
epochs = 10
checkpoint_filename = "temp.h5"
input_shape = (len(X_train_val[0]),)
start_window = 0
end_window = 770


### Model #1 
- Increased neurons to first dense layer, added extra hidden layer. 
- Total parameter count = 3,570,492 (about 6x more than baseline)

<img src="../images/explore_model_1.png" alt="alt text" width="75%">

In [103]:
"""Build model"""
sr = int((end_window-start_window)*2)
hn = sr * 2

model = Sequential()
model.add(
    Dense(int(hn/2), 
        activation='relu', 
        kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4),
        input_shape=(sr,)
    )
)
model.add(BatchNormalization())
model.add(
    Dense(int(hn/4),
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4)
    )
)
model.add(BatchNormalization())
model.add(
    Dense(2, 
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4)
        )
)

print(model.summary())

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 1540)              2373140   
                                                                 
 batch_normalization_39 (Bat  (None, 1540)             6160      
 chNormalization)                                                
                                                                 
 dense_63 (Dense)            (None, 770)               1186570   
                                                                 
 batch_normalization_40 (Bat  (None, 770)              3080      
 chNormalization)                                                
                                                                 
 dense_64 (Dense)            (None, 2)                 1542      
                                                                 
Total params: 3,570,492
Trainable params: 3,565,872
N

### Train the model 

In [104]:
"""Train"""
checkpoint_filename = 'explore_model_1.h5'
callbacks = [
        ModelCheckpoint(
        checkpoint_filename,
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        save_freq="epoch",
    ),
    ReduceLROnPlateau(patience=5, min_delta=1**-6)
]

opt = Adam(learning_rate=init_learning_rate)
model.compile(
    optimizer=opt, 
    loss=CategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

history = model.fit(
    X_train_val, 
    y_train_val, 
    batch_size=batch_size,
    epochs=epochs, 
    validation_split=0.05,   # 45,000 samples
    shuffle=True, 
    callbacks=callbacks,
)

Epoch 1/10
105/105 [==============================] - 18s 159ms/step - loss: 0.4073 - accuracy: 0.9560 - val_loss: 0.3929 - val_accuracy: 0.9564 - lr: 0.0010
Epoch 2/10
105/105 [==============================] - 17s 159ms/step - loss: 0.2997 - accuracy: 0.9643 - val_loss: 0.2823 - val_accuracy: 0.9602 - lr: 0.0010
Epoch 3/10
105/105 [==============================] - 17s 160ms/step - loss: 0.2370 - accuracy: 0.9653 - val_loss: 0.2520 - val_accuracy: 0.9598 - lr: 0.0010
Epoch 4/10
105/105 [==============================] - 17s 158ms/step - loss: 0.1991 - accuracy: 0.9661 - val_loss: 0.2551 - val_accuracy: 0.9602 - lr: 0.0010
Epoch 5/10
105/105 [==============================] - 17s 159ms/step - loss: 0.1687 - accuracy: 0.9685 - val_loss: 0.2851 - val_accuracy: 0.9595 - lr: 0.0010
Epoch 6/10
105/105 [==============================] - 17s 159ms/step - loss: 0.1549 - accuracy: 0.9708 - val_loss: 0.3079 - val_accuracy: 0.9592 - lr: 0.0010
Epoch 7/10
105/105 [==============================] 

In [105]:
"""Load checkpoint"""
co = {}
_add_supported_quantized_objects(co)
model = load_model(checkpoint_filename, custom_objects=co, compile=False)

In [106]:
"""Evaluate"""
y_pred = model.predict(X_test)
print("Test Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))))

model_1_fidelity = compute_fidelity(X_data=X_test, y_data=y_test)
print('Test Fidelity:', model_1_fidelity)

3125/3125 [==============================] - 12s 4ms/step
Test Accuracy: 0.96131
1563/1563 [==============================] - 6s 4ms/step
Total correct: 47258
Total incorrect: 2742
Total samples: 50000
Keras Excited Accuracy: 0.94516
1563/1563 [==============================] - 6s 4ms/step
Total correct: 48873
Total incorrect: 1127
Total samples: 50000
Keras Ground Accuracy: 0.97746
Test Fidelity: 0.92262


### Model #2
- Increased neurons to first dense layer, added 2 extra hidden layer. 
- Total parameter count = 4,457,917 (about 7.5x more than baseline)

<img src="../images/explore_model_2.png" alt="alt text" width="50%">

In [107]:
"""Build model"""
sr = int((end_window-start_window)*2)
hn = sr * 2

model = Sequential()
model.add(
    Dense(int(hn/2), 
        activation='relu',
        input_shape=(sr,), 
        kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4)
))
model.add(BatchNormalization())
model.add(
    Dense(int(hn/4), 
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4)
))
model.add(BatchNormalization())
model.add(
    Dense(int(hn/4), 
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4)
))
model.add(BatchNormalization())
model.add(
    Dense(int(hn/8), 
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4)
))
model.add(BatchNormalization())
model.add(
    Dense(2, 
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.L2(l2=1e-4)
))

print(model.summary())

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)            (None, 1540)              2373140   
                                                                 
 batch_normalization_41 (Bat  (None, 1540)             6160      
 chNormalization)                                                
                                                                 
 dense_66 (Dense)            (None, 770)               1186570   
                                                                 
 batch_normalization_42 (Bat  (None, 770)              3080      
 chNormalization)                                                
                                                                 
 dense_67 (Dense)            (None, 770)               593670    
                                                                 
 batch_normalization_43 (Bat  (None, 770)            

In [108]:
"""Train"""
checkpoint_filename = 'explore_model_2.h5'
callbacks = [
        ModelCheckpoint(
        checkpoint_filename,
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        save_freq="epoch",
    ),
    ReduceLROnPlateau(patience=5, min_delta=1**-6)
]

opt = Adam(learning_rate=init_learning_rate)
model.compile(
    optimizer=opt, 
    loss=CategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

history = model.fit(
    X_train_val, 
    y_train_val, 
    batch_size=batch_size,
    epochs=epochs, 
    validation_split=0.05,   # 45,000 samples
    shuffle=True, 
    callbacks=callbacks,
)

Epoch 1/10
105/105 [==============================] - 24s 215ms/step - loss: 0.5193 - accuracy: 0.9565 - val_loss: 0.5729 - val_accuracy: 0.8791 - lr: 0.0010
Epoch 2/10
105/105 [==============================] - 23s 216ms/step - loss: 0.3846 - accuracy: 0.9639 - val_loss: 0.3447 - val_accuracy: 0.9606 - lr: 0.0010
Epoch 3/10
105/105 [==============================] - 23s 216ms/step - loss: 0.2974 - accuracy: 0.9645 - val_loss: 0.2911 - val_accuracy: 0.9590 - lr: 0.0010
Epoch 4/10
105/105 [==============================] - 23s 216ms/step - loss: 0.2474 - accuracy: 0.9650 - val_loss: 0.2660 - val_accuracy: 0.9589 - lr: 0.0010
Epoch 5/10
105/105 [==============================] - 23s 218ms/step - loss: 0.2179 - accuracy: 0.9661 - val_loss: 0.2567 - val_accuracy: 0.9595 - lr: 0.0010
Epoch 6/10
105/105 [==============================] - 22s 214ms/step - loss: 0.1981 - accuracy: 0.9688 - val_loss: 0.2837 - val_accuracy: 0.9594 - lr: 0.0010
Epoch 7/10
105/105 [==============================] 

In [109]:
"""Load best checkpoint"""
co = {}
_add_supported_quantized_objects(co)
model = load_model(checkpoint_filename, custom_objects=co, compile=False)

In [110]:
"""Evaluate"""
y_pred = model.predict(X_test)
print("Test Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))))

model_2_fidelity = compute_fidelity(X_data=X_test, y_data=y_test)
print('Test Fidelity:', model_2_fidelity)

3125/3125 [==============================] - 15s 5ms/step
Test Accuracy: 0.96122
1563/1563 [==============================] - 8s 5ms/step
Total correct: 47294
Total incorrect: 2706
Total samples: 50000
Keras Excited Accuracy: 0.94588
1563/1563 [==============================] - 7s 5ms/step
Total correct: 48828
Total incorrect: 1172
Total samples: 50000
Keras Ground Accuracy: 0.97656
Test Fidelity: 0.92244
